In [ ]:
pip install nltk

In [ ]:
pip install tflearn

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import tflearn
import random
import json
import pickle
import string
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
stemmer = LancasterStemmer()

In [ ]:
with open("greetings.json") as file:
  data = json.load(file)

with open("coding1.json") as file:
  data["intents"].extend(json.load(file)["intents"])

with open("coding2.json") as file:
  data["intents"].extend(json.load(file)["intents"])

with open("coding3.json") as file:
  data["intents"].extend(json.load(file)["intents"])

with open("collegequeries.json") as file:
  data["intents"].extend(json.load(file)["intents"])


In [ ]:
try:
  with open("data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)

except:
  words =[]
  labels = []
  train_x = []
  train_y = []

  for intent in data['intents']:
    for pattern in intent["patterns"]:
      # do stemming -- bring the sentence to root word(s)
      wrds = nltk.word_tokenize(pattern)
      words.extend(wrds)

      # add pattern of words
      train_x.append(wrds)
      train_y.append(intent["tag"])

    if intent["tag"] not in labels:
      labels.append(intent["tag"])

  # delete duplicates
  words = [stemmer.stem(w.translate(str.maketrans('', '', string.punctuation)).lower()) for w in words if w.translate(str.maketrans('', '', string.punctuation)).isalpha()]
  re_puncts = re.compile('[%s]'% re.escape(string.punctuation))
  words = [re_puncts.sub('', w) for w in words]
  words = [w for w in words if w.isalpha()]
  words = sorted(set(words))
  training = []
  output = []
  out_empty = [0 for _ in range(len(labels))]

  for i, xi in enumerate(train_x):
    bag = []

    wrds = [stemmer.stem(w) for w in xi]
    for w in words:
      if w in wrds:
        bag.append(1)
      else :
        bag.append(0)

    output_row = out_empty.copy()
    output_row[labels.index(train_y[i])] = 1

    training.append(bag)
    output.append(output_row)

  training = np.array(training)
  output = np.array(output)

  with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

In [ ]:
len(output)

262

In [ ]:
labels

In [18]:
# ------------- modelling ------------
try:
  model.load("model.tflearn")

except:
  ops.reset_default_graph()
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)
  net = tflearn.input_data(shape=[None, len(training[0])])
  net = tflearn.fully_connected(net, 8) # hidden layer
  net = tflearn.fully_connected(net, 8) # hidden layer
  net = tflearn.fully_connected(net, len(output[0]), activation="softmax") # output layer
  net = tflearn.regression(net)

  model = tflearn.DNN(net)
  model.fit(training, output, n_epoch=500, batch_size=10, show_metric=True)
  # model.summary()
  model.save("model.tflearn")

Training Step: 13499  | total loss: 0.00681 | time: 0.151s
| Adam | epoch: 500 | loss: 0.00681 - acc: 0.9950 -- iter: 260/262
Training Step: 13500  | total loss: 0.00617 | time: 0.156s
| Adam | epoch: 500 | loss: 0.00617 - acc: 0.9955 -- iter: 262/262
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def vectorize(x, words):
  vector = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(x)
  s_words = [stemmer.stem(w.lower()) for w in s_words]
  re_puncts = re.compile('[%s]'% re.escape(string.punctuation))
  s_words = [re_puncts.sub('', w) for w in s_words]
  s_words = [w for w in s_words if w.isalpha()]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        vector[i] = 1

  return np.array(vector)

In [ ]:
def demo_chat():
  '''
  temporary function for local trial of chatbot
  '''
  print("Chatter away")

  while True:
    inp = input("You : ")
    if inp.lower() == "break" :
      break
    
    results = model.predict([vectorize(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]
    
    if results[0][results_index] > 0.7:
      for tg in data["intents"]:
        if tg["tag"] == tag:
          responses = tg["responses"]

      print(random.choice(responses))

    else:
      print("Are you messing with me?")

In [ ]:
demo_chat()

Chatter away
You : break
